# Examples of Forte inputs

## Computing one or more FCI solutions of a given symmetry

The first example shows how to perform separate computations on different electronic
states of methylene. We compute the lowest $^3B_1$ state and the first two $^1A_1$ states.
All of these computations use ROHF orbitals optimized for the lowest $^3B_1$.
Here we use the python API interface of Forte, but it is easy to translate
these examples to a psi4 psithon input.

This input (see `examples/api/02_rohf_fci/test_02_rohf_fci.py`) starts with the geometry specification:
```python
# examples/api/02_rohf_fci/input.py

import psi4
import forte

mol = psi4.geometry("""
0 3 # triplet state
C
H 1 1.085
H 1 1.085 2 135.5
""")

psi4.set_options(
    {
        'basis': 'DZ',
        'scf_type': 'pk', # <-- request conventional two-electron integrals
        'e_convergence': 12,
        'reference': 'rohf',
        'forte__active_space_solver': 'fci',
        'forte__restricted_docc': [1, 0, 0, 0],
        'forte__active': [3, 0, 2, 2],
        'forte__multiplicity': 3, # <-- triplet state
        'forte__root_sym': 2, # <-- B1 symmetry
    }
)
```
Note that all options prefixed with `forte__` are specific to the Forte computation.
Here we specify a multiplicity equal to 3 and the $B_1$ irrep (`root_sym = 2`).
In this example we keep the lowest $A_1$ MO doubly occupied
(`'restricted_docc': [1, 0, 0, 0]`) and use an active space that contains three
$A_1$ MOs, and two MOs each of $B_1$ and $B_2$ symmetry. Lastly, we run Forte:
```python
    psi4.energy('forte')
```
This input will run a CASCI computation (since we have not requested orbital optimization).
An example of how to request orbital optimization can be found in the section
*Computing a manifold of solutions of different symmetry*.
The output will return the energy and show the composition of the wave function:

    ==> Root No. 0 <==

    2b0 a0 20      0.70326213
    2a0 b0 20     -0.70326213

    Total Energy:     -38.924726774489, <S^2>: 2.000000

    ==> Energy Summary <==

    Multi.(2ms)  Irrep.  No.               Energy      <S^2>
    --------------------------------------------------------
       3  (  0)    B1     0      -38.924726774489   2.000000
    --------------------------------------------------------

Next we change the options to compute the lowest two $1A_1$ states.
We modify the multiplicity, the symmetry, and indicate that we want two
roots (`NROOTS = 2`):
```python
    psi4.set_options({
        'forte__multiplicity': 1,
        'forte__root_sym': 0, # <-- A1 symmetry
        'forte__nroots' : 2}
    )
    psi4.energy('forte')
```
The results of this computation is:

    ==> Root No. 0 <==

    220 00 20      0.92134189
    200 20 20     -0.37537841

    Total Energy:     -38.866616413802, <S^2>: -0.000000

    ==> Root No. 1 <==

    200 20 20     -0.89364609
    220 00 20     -0.36032959
    ab0 20 20     -0.13675846
    ba0 20 20     -0.13675846

    Total Energy:     -38.800424868719, <S^2>: -0.000000

    ==> Energy Summary <==

    Multi.(2ms)  Irrep.  No.               Energy      <S^2>
    --------------------------------------------------------
       1  (  0)    A1     0      -38.866616413802  -0.000000
       1  (  0)    A1     1      -38.800424868719  -0.000000
    --------------------------------------------------------

## State-averaged CASSCF with states of different symmetry

The next example shows how to perform a state-averaged CASSCF computation on two
electronic states of different symmetries. We still consider methylene, and average the lowest $^3B_1$ and $^1A_1$ states.
To begin, we use ROHF orbitals optimized for the lowest $^3B_1$. However,
the final orbitals will optimize the average energy
\begin{equation}
E_\mathrm{avg} = \frac{1}{2} \left(E_{^3B_1} + E_{^1A_1}\right).
\end{equation}
We use the same active space of the previous example, but here to specify the state, we set the `AVG_STATE` option:
```python
# examples/api/03_sa-casscf.py

import psi4
import forte

psi4.geometry("""
0 3
C
H 1 1.085
H 1 1.085 2 135.5
""")

psi4.set_options({'basis': 'DZ', 'scf_type': 'pk', 'e_convergence': 12, 'reference': 'rohf',
        'forte__job_type': 'mcscf_two_step',
        'forte__active_space_solver': 'fci',
        'forte__restricted_docc': [1, 0, 0, 0],
        'forte__active': [3, 0, 2, 2],
        'forte__avg_state': [[2, 3, 1], [0, 1, 1]]
        # [(B1, triplet, 1 state), (A1,singlet,1 state)]
    }
)

psi4.energy('forte')
```

The output of this computation (in `examples/api/03_sa-casscf.out`) shows the energy for both states in the following table:
```
    ==> Energy Summary <==

    Multi.(2ms)  Irrep.  No.               Energy      <S^2>
    --------------------------------------------------------
       1  (  0)    A1     0      -38.900217662950   0.000000
    --------------------------------------------------------
       3  (  0)    B1     0      -38.960623289646   2.000000
    --------------------------------------------------------
```